In [1]:
import folium
from folium import Marker
from folium.plugins import MarkerCluster
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image
from matplotlib import font_manager,rc

import random

%matplotlib inline
from matplotlib import font_manager,rc
# 한글 폰트
font_location = "C:\Windows\Fonts\malgun.ttf"
font_name = font_manager.FontProperties(fname=font_location).get_name()
rc('font',family=font_name)
plt.rcParams['axes.unicode_minus'] = False

## 주요절차

### 1. 초기데이터 분석

#### 1-1. 데이터딕셔너리(각 열의 의미, 데이터의 의미해석관련 스펙 또는 메뉴얼) 확인
#### 1-2 데이터 기초정보 확인 (사이즈, 열이름, 데이터형)

In [34]:
df[df['건축년도'] > 2000].info()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 33 to 624
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시도명       13 non-null     object 
 1   시군구명      13 non-null     object 
 2   읍면동명      13 non-null     object 
 3   주택유형      13 non-null     object 
 4   건축년도      13 non-null     int64  
 5   건축물대장주구조  13 non-null     object 
 6   연면적       13 non-null     float64
 7   위도        13 non-null     float64
 8   경도        13 non-null     float64
 9   등급판정결과    13 non-null     object 
 10  데이터기준일자   13 non-null     object 
dtypes: float64(3), int64(1), object(7)
memory usage: 1.2+ KB


In [2]:
df = pd.read_csv('광주광역시 동구_빈집 현황_20250716.csv',encoding='cp949')
df.head()

,시도명,시군구명,읍면동명,주택유형,건축년도,건축물대장주구조,연면적,위도,경도,등급판정결과,데이터기준일자
0,광주광역시,동구,대인동,그외,0,벽돌구조,139.820,192083.0095,183919.9701,2등급,2025-07-16
1,광주광역시,동구,대인동,단독,0,일반목구조,98.115,192160.5924,183779.9067,2등급,2025-07-16
2,광주광역시,동구,대인동,단독,1965,일반목구조,39.660,192436.9433,183844.8592,2등급,2025-07-16
3,광주광역시,동구,금남로5가,단독,1981,벽돌구조,176.320,191854.7955,183687.0436,1등급,2025-07-16
4,광주광역시,동구,궁동,단독,1968,벽돌구조,148.080,192572.6725,183428.8097,2등급,2025-07-16


In [8]:
df = df [ df['건축년도'] != 0 ]
df

,시도명,시군구명,읍면동명,주택유형,건축년도,건축물대장주구조,연면적,위도,경도,등급판정결과,데이터기준일자
2,광주광역시,동구,대인동,단독,1965,일반목구조,39.660,192436.9433,183844.8592,2등급,2025-07-16
3,광주광역시,동구,금남로5가,단독,1981,벽돌구조,176.320,191854.7955,183687.0436,1등급,2025-07-16
4,광주광역시,동구,궁동,단독,1968,벽돌구조,148.080,192572.6725,183428.8097,2등급,2025-07-16
5,광주광역시,동구,궁동,단독,1984,벽돌구조,125.880,192502.8990,183391.0862,1등급,2025-07-16
6,광주광역시,동구,장동,단독,1951,일반목구조,68.410,192857.2541,183515.7902,2등급,2025-07-16
...,...,...,...,...,...,...,...,...,...,...,...
642,광주광역시,동구,용산동,단독,1971,블록구조,78.330,193550.3959,179946.3202,2등급,2025-07-16
643,광주광역시,동구,용산동,단독,1978,벽돌구조,75.245,193650.0085,180214.5502,1등급,2025-07-16
645,광주광역시,동구,용산동,단독,1976,벽돌구조,52.550,193675.3286,180110.5382,1등급,2025-07-16
646,광주광역시,동구,용산동,단독,1977,벽돌구조,97.850,193802.2706,180228.8692,2등급,2025-07-16


* 기초통계정보 확인

### 2. 데이터 정제

#### 2.1. 원본데이터에서 분석가능한 형태로 데이터 전처리 (형변환, 열이름 변경등)
#### 2.2. 데이터 클리닝: 이상치, 결측치처리

In [16]:
df2 = df[['읍면동명', '주택유형', '건축년도', '건축물대장주구조', '위도', '경도', '등급판정결과']]
df.head()

,시도명,시군구명,읍면동명,주택유형,건축년도,건축물대장주구조,연면적,위도,경도,등급판정결과,데이터기준일자
2,광주광역시,동구,대인동,단독,1965,일반목구조,39.66,192436.9433,183844.8592,2등급,2025-07-16
3,광주광역시,동구,금남로5가,단독,1981,벽돌구조,176.32,191854.7955,183687.0436,1등급,2025-07-16
4,광주광역시,동구,궁동,단독,1968,벽돌구조,148.08,192572.6725,183428.8097,2등급,2025-07-16
5,광주광역시,동구,궁동,단독,1984,벽돌구조,125.88,192502.8990,183391.0862,1등급,2025-07-16
6,광주광역시,동구,장동,단독,1951,일반목구조,68.41,192857.2541,183515.7902,2등급,2025-07-16


### 3. 추가데이터 분석

#### 3.1 데이터 정제후 데이터딕셔너리 재확인
#### 3.2 기초통계 정보 확인 (최대값, 최소값, 평균값, 표준편차, 최빈값, 유일값 등)
#### 3.3 시각화를 통한 데이터이해 심화 

In [17]:
df2.head(10)

,읍면동명,주택유형,건축년도,건축물대장주구조,위도,경도,등급판정결과
2,대인동,단독,1965,일반목구조,192436.9433,183844.8592,2등급
3,금남로5가,단독,1981,벽돌구조,191854.7955,183687.0436,1등급
4,궁동,단독,1968,벽돌구조,192572.6725,183428.8097,2등급
5,궁동,단독,1984,벽돌구조,192502.8990,183391.0862,1등급
6,장동,단독,1951,일반목구조,192857.2541,183515.7902,2등급
7,장동,단독,1959,일반목구조,192805.1265,183514.0367,2등급
8,장동,단독,1965,일반목구조,192875.6587,183440.3522,1등급
9,장동,단독,1993,벽돌구조,192886.6518,183164.2227,2등급
10,장동,단독,1979,벽돌구조,192877.8488,183171.7282,2등급
11,동명동,단독,1956,일반목구조,193574.9859,183428.8021,2등급


In [10]:
df.describe()

,건축년도,연면적,위도,경도
count,629.000000,629.000000,629.000000,629.000000
mean,1971.292528,72.678149,193254.938814,183399.827460
std,11.193874,43.942710,657.188967,1118.976681
min,1920.000000,0.000000,191854.795500,179946.320200
25%,1964.000000,47.590000,192600.529900,183033.758200
50%,1971.000000,62.890000,193293.109100,183793.835700
75%,1976.000000,84.950000,193767.150000,184144.174200
max,2011.000000,502.880000,195667.894400,185082.863700


In [21]:
df['건축물대장주구조'].value_counts()

건축물대장주구조
벽돌구조        330
일반목구조       246
블록구조         35
목구조           8
철근콘크리트구조      6
경량철골구조        3
시멘트블럭조        1
Name: count, dtype: int64

In [25]:
df['읍면동명'].value_counts()

읍면동명
계림동      158
산수동      148
지산동       81
동명동       74
소태동       61
학동        36
금동        30
서석동       10
남동         6
용산동        5
운림동        5
장동         5
불로동        5
궁동         2
대인동        1
금남로5가      1
호남동        1
Name: count, dtype: int64

In [24]:
df['건축년도'].value_counts()

건축년도
1964    47
1971    39
1974    39
1973    34
1972    34
        ..
1920     1
1921     1
1998     1
1945     1
1996     1
Name: count, Length: 62, dtype: int64

In [34]:
df[df['건축년도'] > 2000].info()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 33 to 624
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시도명       13 non-null     object 
 1   시군구명      13 non-null     object 
 2   읍면동명      13 non-null     object 
 3   주택유형      13 non-null     object 
 4   건축년도      13 non-null     int64  
 5   건축물대장주구조  13 non-null     object 
 6   연면적       13 non-null     float64
 7   위도        13 non-null     float64
 8   경도        13 non-null     float64
 9   등급판정결과    13 non-null     object 
 10  데이터기준일자   13 non-null     object 
dtypes: float64(3), int64(1), object(7)
memory usage: 1.2+ KB


In [35]:
df[df['건축년도'] > 1980].info()

<class 'pandas.core.frame.DataFrame'>
Index: 97 entries, 3 to 647
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시도명       97 non-null     object 
 1   시군구명      97 non-null     object 
 2   읍면동명      97 non-null     object 
 3   주택유형      97 non-null     object 
 4   건축년도      97 non-null     int64  
 5   건축물대장주구조  97 non-null     object 
 6   연면적       97 non-null     float64
 7   위도        97 non-null     float64
 8   경도        97 non-null     float64
 9   등급판정결과    97 non-null     object 
 10  데이터기준일자   97 non-null     object 
dtypes: float64(3), int64(1), object(7)
memory usage: 9.1+ KB


In [23]:
df['주택유형'].value_counts()

주택유형
단독     621
그외       5
다가구      3
Name: count, dtype: int64

In [22]:
df['등급판정결과'].value_counts()

등급판정결과
2등급    375
1등급    209
3등급     45
Name: count, dtype: int64

In [47]:
def group_year(year):
    if year > 1980 :
        return "a" 
    elif year > 1970 :
        return "b" 
    elif year > 1960 :
        return "c"
    elif year > 1950 :
        return "d" 
    else : return "e"


df["연"] = df["건축연도"].apply(group_year)

KeyError: '건축연도'

In [46]:
df['연'].value_counts()

연
b    248
c    212
a     97
d     65
e      7
Name: count, dtype: int64

#### 3.1 데이터 정제후 데이터딕셔너리 재확인
#### 3.2 기초통계 정보 확인 (최대값, 최소값, 평균값, 표준편차, 최빈값, 유일값 등)
#### 3.3 시각화를 통한 데이터이해 심화 

* 3.3.1 단일 열: 데이터의 성격에 따른 다양한 차트 적용  
* 3.3.2 복수 열: 두개의 열간 연관관계 분석(Pair Plot 또는 Seaborn의 관련 지원 차트등)

#### 3.4 데이터 분석 결과도출

# 데이터분석 적용 실습 템플릿

## 데이터 수집

In [1]:
# 데이터 소스를 확보하세요


## 1. 초기데이터 분석

### 1-1. 데이터스키마 확인

#### 각 열의 의미, 데이터의 의미해석관련 스펙 또는 메뉴얼을 확인 가능시 진행 하세요
* A열: A열의 의미를 기술
* B열: B열의 의미를 기술  
...

### 1-2. 데이터 기초정보 확인

* 사이즈, 데이터형, 기초통계항목등을 확인하세요  

In [ ]:
#df.shape
#df.info()

* 기초통계항목

In [ ]:
# df.desribe()

## 2. 데이터 정제

### 2-1. 간단한 데이터 전처리

In [5]:
# 원본데이터로에서 읽기 가능한 형태로 데이터처리를 하세요. (열이름 변경, 형변환 등)

### 2-2. 데이터 클리닝

In [6]:
# 결측치, 중복값, 이상치 처리등의 작업을 진행하세요

* 결측치

    - 결측치 열 확인

In [ ]:
# missing_values = df.isnull().any()  
# column_missing_values = missing_values[ missing_values == True ]
# column_missing_values

In [ ]:
# df.isnull().any(axis=1)

* 결측치 보정

In [ ]:
# df['나이'] = df['나이'].fillna(0)

In [1]:
# df = df.fillna(0)

* 중복값 삭제 (필요시)

In [ ]:
# df[df.duplicated(keep=False)]

In [ ]:
# df.drop_duplicates(inplace=True)

## 3. 추가데이터 분석

### 3.1. 데이터 정제후 데이터스키마 재확인

* 1-2 작업을 다시 해보세요.

### 3.2 데이터 기초정보 정보 확인

* 최대값, 최소값, 평균값, 표준편차, 최빈값, 유일값, 결측치, 이상치 등의 정보를 재확인해 보세요

### 3.3 시각화를 통한 데이터이해 심화

* 데이터의 성격에 따라 이해도를 올리기 위해 시각화가 필요한 데이터차트를 만들어 보세요.  
    * (Seaborn 추천: 페어플롯, 막대차트, 파이차트, 라인차트, 히스토그램등..)
    * 차트 분석시 hue인자를 활용해서 범주형 열에 대한 추가 분석을 해보세요.
* 시각화한 차트에 대한 분석

### 3.4 최종 데이터 분석 결과도출

* 데이터 분석을 통해 얻은 새로운 지식, 인사이트를 정리하세요.
* 새로운 지식 및 인사이트 별 비지니스 전략을 도출할 수 있으면 정리해 보세요